In [1]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup

In [2]:
url = "https://myanimelist.net/topanime.php"
top = 50 #para limitar a lista

# LENDO A TABELA

In [3]:
anime_list = pd.read_html(url,header=0)[0] #Pega a tabela
anime_list = anime_list.drop(columns=["Your Score","Status"]) #Apaga as colunas que n vai precisar
anime_list = anime_list[:top] #limita a lista usando o valor da vaiavel top animes
anime_list

,Rank,Title,Score
0,1,Fullmetal Alchemist: Brotherhood TV (64 eps) ...,9.10
1,2,Steins;Gate TV (24 eps) Apr 2011 - Sep 2011 ...,9.07
2,3,Bleach: Sennen Kessen-hen TV (13 eps) Oct 20...,9.06
3,4,GintamaÂ° TV (51 eps) Apr 2015 - Mar 2016 6...,9.06
4,5,Shingeki no Kyojin Season 3 Part 2 TV (10 eps...,9.05
5,6,Shingeki no Kyojin: The Final Season - Kankets...,9.04
6,7,Gintama' TV (51 eps) Apr 2011 - Mar 2012 53...,9.04
7,8,Gintama: The Final Movie (1 eps) Jan 2021 - ...,9.04
8,9,Hunter x Hunter (2011) TV (148 eps) Oct 2011...,9.04
9,10,Kaguya-sama wa Kokurasetai: Ultra Romantic TV...,9.04


# ALTERANDO E CRIANDO AS COLUNAS A PARTIR DO TITLE (separando usando regex)

In [4]:
df_anime = pd.DataFrame(anime_list)
df_anime["Title"] =  [re.search(r"(\w.*)(TV|Movie|OVA|ONA|Special)",anime).group(1) for anime in anime_list["Title"]]
df_anime["Type"] =  [re.search(r"(\w.*)(TV|Movie|OVA|ONA|Special)",anime).group(2) for anime in anime_list["Title"]]
df_anime["Episodes"] =[re.search(r"(\d+)? eps",anime).group().strip("eps") for anime in anime_list["Title"]]
df_anime["Years"] = [re.search(r"(\w+ \d{4}) [-] (\w+ \d{4})?",anime).group() for anime in anime_list["Title"]]
df_anime["Members"] =  [re.search(r"(\d+,?\d+?,?\d+? members)",anime).group().strip("members") for anime in anime_list["Title"]]
df_anime

,Rank,Title,Score,Type,Episodes,Years,Members
0,1,Fullmetal Alchemist: Brotherhood,9.10,TV,64,Apr 2009 - Jul 2010,"3,204,276"
1,2,Steins;Gate,9.07,TV,24,Apr 2011 - Sep 2011,"2,462,605"
2,3,Bleach: Sennen Kessen-hen,9.06,TV,13,Oct 2022 - Dec 2022,"464,356"
3,4,GintamaÂ°,9.06,TV,51,Apr 2015 - Mar 2016,"602,082"
4,5,Shingeki no Kyojin Season 3 Part 2,9.05,TV,10,Apr 2019 - Jul 2019,"2,131,956"
5,6,Shingeki no Kyojin: The Final Season - Kankets...,9.04,Special,2,Mar 2023 -,"460,936"
6,7,Gintama',9.04,TV,51,Apr 2011 - Mar 2012,"531,302"
7,8,Gintama: The Final,9.04,Movie,1,Jan 2021 - Jan 2021,"135,789"
8,9,Hunter x Hunter (2011),9.04,TV,148,Oct 2011 - Sep 2014,"2,686,232"
9,10,Kaguya-sama wa Kokurasetai: Ultra Romantic,9.04,TV,13,Apr 2022 - Jun 2022,"842,043"


# RECEBENDO E FILTRANDO OS LINKS DOS ANIMES

In [5]:
pagina = requests.get(url) # instanciando os dados da pagina

sopa = BeautifulSoup(pagina.content,"html.parser") # iniciando com o BS4

bloco_tabela = sopa.find("table") # procura a tabela

links = bloco_tabela.find_all("a",class_="hoverinfo_trigger fl-l ml12 mr8" ,href=True) # procurando a tag que esta o link na tabela
links = [link["href"] for link in links] # armazenando cada link numa lista

links=links[:top] #limitando a lista para o valor escolhido

# ENTRANDO EM CADA LINK PARA GUARDAR AS INFORMAÇÕES

In [6]:
matriz_etaria, matriz_genero = [[],[]] #criandp as duas lista vazias para inserir os dados da matriz

# Looping para ler cada link na lista receber os dados, adicionar na lista e ir para o proximo link
for url in links:
    pagina_anime = requests.get(url)
    body = BeautifulSoup(pagina_anime.content,"html.parser")
    bloco_side = body.find("div",class_="leftside")

    #corpo
    body = BeautifulSoup(pagina_anime.content,"html.parser")
    bloco_side = body.find("div",class_="leftside")


    try:
        #genero
        generos = bloco_side.find_all("span",attrs={"itemprop": "genre"})
        generos = [genero.get_text() for genero in generos]

        #idade
        faixa_etaria = bloco_side.find(string=re.compile(r"\s[A-Za-z]{1,2}\s?-|[+]\s?")).text
        faixa_etaria = re.search(r"(\w.+?).+(-|[+])",faixa_etaria).group()
    except:
        generos = ""
        faixa_etaria = ""
    #dicionario
    matriz_genero.append(", ".join(str(genero) for genero in generos))
    matriz_etaria.append(faixa_etaria)

# ADICIONANDO AO DF_ANIME

In [7]:
df_anime["Generos"] = matriz_genero #criando a coluna generos e adicionando os dados
df_anime["Faixa etaria"] = matriz_etaria #criando a coluna generos e adicionando os dados

In [8]:
df_anime

,Rank,Title,Score,Type,Episodes,Years,Members,Generos,Faixa etaria
0,1,Fullmetal Alchemist: Brotherhood,9.10,TV,64,Apr 2009 - Jul 2010,"3,204,276","Action, Adventure, Drama, Fantasy, Military, S...",R - 17+
1,2,Steins;Gate,9.07,TV,24,Apr 2011 - Sep 2011,"2,462,605","Drama, Sci-Fi, Suspense, Psychological, Time T...",PG-13 -
2,3,Bleach: Sennen Kessen-hen,9.06,TV,13,Oct 2022 - Dec 2022,"464,356","Action, Adventure, Fantasy, Shounen",R - 17+
3,4,GintamaÂ°,9.06,TV,51,Apr 2015 - Mar 2016,"602,082","Action, Comedy, Sci-Fi, Gag Humor, Historical,...",PG-13 -
4,5,Shingeki no Kyojin Season 3 Part 2,9.05,TV,10,Apr 2019 - Jul 2019,"2,131,956","Action, Drama, Gore, Military, Survival, Shounen",R - 17+
5,6,Shingeki no Kyojin: The Final Season - Kankets...,9.04,Special,2,Mar 2023 -,"460,936","Action, Drama, Suspense, Gore, Military, Survi...",R - 17+
6,7,Gintama',9.04,TV,51,Apr 2011 - Mar 2012,"531,302","Action, Comedy, Sci-Fi, Gag Humor, Historical,...",PG-13 -
7,8,Gintama: The Final,9.04,Movie,1,Jan 2021 - Jan 2021,"135,789","Action, Comedy, Drama, Sci-Fi, Gag Humor, Hist...",PG-13 -
8,9,Hunter x Hunter (2011),9.04,TV,148,Oct 2011 - Sep 2014,"2,686,232","Action, Adventure, Fantasy, Shounen",PG-13 -
9,10,Kaguya-sama wa Kokurasetai: Ultra Romantic,9.04,TV,13,Apr 2022 - Jun 2022,"842,043","Comedy, Romance, School, Seinen",PG-13 -
